In [111]:
# https://pypi.org/project/torch-geometric/
# pip install torch
# pip install torch-scatter -f https://data.pyg.org/whl/torch-1.13.0+cpu.html
# pip install torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cpu.html
# pip install torch-geometric


# https://towardsdatascience.com/louvains-algorithm-for-community-detection-in-python-95ff7f675306
import torch
from torch import Tensor
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from collections import Counter 

In [2]:
# plot the data
from utils.visual import plot

#plot(0) 

In [3]:
import re

def find_train_ego_index():
    pre, pree = [], []
    for item in train.files:
        pre.append(int(re.sub('\D', '',item )))

    for i in egonet.files:
        pree.append(int(re.sub('\D', '',i )))
        
    train_ego_index = []
    for i in range(len(pree)):
        if pree[i] in pre:
            train_ego_index.append(i)
    return train_ego_index, pree, pre

In [4]:
# self-made packages
from utils.data import data_preprocess


train = data_preprocess("\Training")
train_list, train_dict = train.train_data()
df = pd.DataFrame(data=train_dict)

In [5]:
egonet = data_preprocess("\egonets")
ego_list, user_list = egonet.ego_data()

In [6]:
# 這個是指應該將哪些 ego_list 用在 training，方便輸入0,1,2，就不用輸入 239、345、661...
train_ego_index,ego_user, train_user  = find_train_ego_index()

## Data Preparation

In [7]:
from utils.user import users

users = users()
index = user_list[train_ego_index[2]] # 更改數字可以挑選要哪一個 training data 

# edge information EX: (240, 241), (240, 242)...
edge1 = egonet.edge_data(train_ego_index[0])

In [8]:
#users = pd.read_csv('df_new_friend.csv') #把註解拿掉代表讀取新的feature

## Predict Data

In [9]:
circlesP = {}

# 這裡是整理所有的 predict data，但因為目前沒有所以暫時用 training 的代替
for i in range(len(train_dict)):
    uidP,friendsP = train_dict[i]['UserId'], train_dict[i]['Predicted'] # predict 
    circlesP[int(uidP)] = [set([int(x) for x in c.split()]) for c in friendsP.split(';')]

## Connected Component
#### 測試connected component

In [10]:
from connected_components import findCommunities
import sys, os

def connected_componet(target_dir):
    egoUser = -1
    try:
        egoUser = int(re.sub("\D", '', target_dir))
    except Exception as e:
        print("Expected files to be names 'X.egonet' where X is a user ID")

    cs = list(findCommunities(target_dir))

    if len(cs) == 0:
        cs = [set(adj.keys())]
    cs = [' '.join([str(y) for y in x]) for x in cs]
    res = str(egoUser) + ',' + ';'.join(cs)
    
    return res
    

In [11]:
def process(test_ego_num):
    return [{
            'UserId': test_ego_num.split(',')[0],
            'Predicted': test_ego_num.split(',')[1]
            }]


In [12]:
connect_egonet = []
for i in range(len(train.files)):
    target_dir = os.getcwd()+"\\egonets\\"+train.files[i].replace('circles', 'egonet')
    test_ego_num = connected_componet(target_dir)
    test = process(test_ego_num)
    connect_egonet+=(test)

# Caculate Loss
### 之後可能要放進一個mode裡面當作 objective function 之類的，這邊先暫時表達一次性的評分

## Model

In [13]:
from sklearn.cluster import KMeans

def exe(n_clusters):
    km = KMeans(
        n_clusters=n_clusters, init='random',
        n_init=10, max_iter=300, 
        tol=1e-04, random_state=2
    )
    final, indexx = [], []
    for i in range(len(train_ego_index)):
        index = user_list[train_ego_index[i]] # 更改數字可以挑選要哪一個 training data 
        test = users.iloc[index]

        X = np.array(test).astype(np.int64)
        return X
        y_km = km.fit(X)
        predict = y_km.predict(X)
        final.append(predict)
        indexx.append(index)
    return final, indexx

In [14]:
from socialCircles_metric import loss1

def temperal(train_dict):
        
    circlesG = {}

    for i in range(len(train_dict)):
        uidG,friendsG = train_dict[i]['UserId'], train_dict[i]['Predicted'] # ground truth
        circlesG[int(uidG)] = [set([int(x) for x in c.split()]) for c in friendsG.split(';')] 
        
        
    totalLoss = 0
    for k in circlesP.keys():
        try:
            if k not in circlesG:
                print("Ground-truth has prediction for circle", k, "but prediction does not")
                break

            l = loss1(circlesG[k], circlesP[k])
            #print("loss for user", k, "=", l)
            totalLoss += l
        except:
            continue
            
    print("total loss for all users =", totalLoss)

In [15]:
def data(n_clusters):

    final, index = exe(n_clusters)
    
    resss = []
    for k in range(len(final)):
        fina = ""
        for i in range(n_clusters):
            tem = ''

            for j in range(len(final[k])):
                if final[k][j] == i:
                    tem+= ' ' + str(index[k][j])
            fina += tem.strip() + ';'
        resss.append(fina)
        
    transform_label = []
    
    for i in range(len(index)):
        transform_label.append({"UserId":index[i][0]-1, "Predicted":resss[i].strip()[:-1]})
        
    return transform_label

In [16]:
temperal(connect_egonet)

total loss for all users = 17606


In [17]:
temperal(train_dict)

total loss for all users = 0


In [18]:
from collections import Counter

def count_similarity(a, b):
    count = 0
    for i in range(len(a)):
        if (a[i] != -1) and (a[i] == b[i]):
            count+=1
            
    return count #, (len(a) - Counter(a)[-1]) # 分母為去除掉 -1 的個數

In [22]:
from socialCircles_metric import loss1

def another_temperal(circlesG):
        
    totalLoss = 0
    for k in circlesP.keys():
        try:
            if k not in circlesG:
                #print("Ground-truth has prediction for circle", k, "but prediction does not")
                #break
                pass

            l = loss1(circlesG[k], circlesP[k])
            print("loss for user", k, "=", l)
            totalLoss += l
        except:
            continue
            
    print("total loss for all users =", totalLoss)

In [109]:
from tqdm import tqdm, trange
import community
from utils.visual import plot, draw, colors
import networkx as nx
import networkx.algorithms.community as nx_comm

t = {}
r = []

for i in trange(len(train_ego_index)):
    index = train_ego_index[i]
    first_user = user_list[index][0]
    test = users.iloc[user_list[index]]
    
    res = []
    for j in range(len(user_list[index])):
        user, friend = ego_list[index][j][0], ego_list[index][j][1:]
        temp1 = []
        for i in range(len(friend)):
            temp1.append(count_similarity(test.iloc[user-first_user], test.iloc[friend[i] - first_user]))

        res.append(temp1)
        
    G = draw(os.path.join(egonet.target_dir, egonet.files[index]))

    for l in range(len(ego_list[index])):
        lis = ego_list[index][l]
        source = lis[0]
        for i in range(len(lis[1:])):
            dest = lis[1:][i]
            G[source][dest]['weight'] = res[l][i]

    aaa = nx_comm.louvain_communities(G,threshold=0.0001 , seed=2)
    
    t[first_user-1] = aaa
    r.append(res)
    

100%|██████████████████████████████████████████████████████████████████████████████████| 56/56 [00:40<00:00,  1.40it/s]


In [110]:
another_temperal(t)

loss for user 239 = 110
loss for user 345 = 210
loss for user 611 = 257
loss for user 1357 = 474
loss for user 1839 = 119
loss for user 1968 = 300
loss for user 2255 = 117
loss for user 2365 = 83
loss for user 2738 = 32
loss for user 2790 = 133
loss for user 2895 = 59
loss for user 3735 = 714
loss for user 4406 = 838
loss for user 4829 = 218
loss for user 5494 = 299
loss for user 5881 = 465
loss for user 6413 = 267
loss for user 6726 = 248
loss for user 7667 = 257
loss for user 8100 = 108
loss for user 8239 = 54
loss for user 8553 = 149
loss for user 8777 = 263
loss for user 9103 = 97
loss for user 9642 = 326
loss for user 9846 = 149
loss for user 9947 = 413
loss for user 10395 = 339
loss for user 10929 = 55
loss for user 11014 = 158
loss for user 11364 = 38
loss for user 11410 = 205
loss for user 12800 = 486
loss for user 13353 = 105
loss for user 15672 = 789
loss for user 16203 = 71
loss for user 16378 = 225
loss for user 16642 = 325
loss for user 16869 = 69
loss for user 17951 = 33


In [34]:
import community
from utils.visual import plot, draw, colors
import networkx as nx
import networkx.algorithms.community as nx_comm

# Create a graph using NetworkX
def do():
    aa=[]
    bb={}
    
    for i in range(len(train_ego_index)):
        index = train_ego_index[i]
        first_user = user_list[index][0]
        temp_path = os.path.join(egonet.target_dir, egonet.files[index]) #51
        G = draw(temp_path)
#         if index == 51:
#             return G
        
        bb[first_user-1] = nx_comm.louvain_communities(G,threshold=0.1 , seed=123)

#         partition = community.best_partition(G)
#         res = sorted(partition.items(), key=lambda item: item[1])
#         aa+=process_res(res, index)
    
    return bb #aa

In [35]:
def process_res(res, first_user):
    temp = ''
    index = 0
    a = ''
    for i in range(len(res)):
        if index != res[i][1]:
            index = res[i][1]
            temp += a.strip() + ';'
            a = ''
        a += ' '+str(res[i][0])
    temp += a.strip()
        
    return [{'UserId':first_user-1, 'Predicted':temp}]

In [36]:
aaa = do()

In [37]:
another_temperal(aaa)

loss for user 239 = 120
loss for user 345 = 224
loss for user 611 = 265
loss for user 1357 = 470
loss for user 1839 = 121
loss for user 1968 = 326
loss for user 2255 = 117
loss for user 2365 = 83
loss for user 2738 = 36
loss for user 2790 = 117
loss for user 2895 = 63
loss for user 3735 = 766
loss for user 4406 = 834
loss for user 4829 = 242
loss for user 5494 = 293
loss for user 5881 = 473
loss for user 6413 = 259
loss for user 6726 = 200
loss for user 7667 = 239
loss for user 8100 = 78
loss for user 8239 = 66
loss for user 8553 = 157
loss for user 8777 = 245
loss for user 9103 = 103
loss for user 9642 = 324
loss for user 9846 = 141
loss for user 9947 = 393
loss for user 10395 = 353
loss for user 10929 = 59
loss for user 11014 = 168
loss for user 11364 = 34
loss for user 11410 = 205
loss for user 12800 = 489
loss for user 13353 = 92
loss for user 15672 = 805
loss for user 16203 = 63
loss for user 16378 = 221
loss for user 16642 = 320
loss for user 16869 = 65
loss for user 17951 = 33
l